In [ ]:
import pickle
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import torch
import tensorflow as tf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#load in the labels
import pandas as pd
labels = pd.read_csv('/content/drive/MyDrive/BioE212/test/rif_labels.csv')

In [ ]:
labels

,Unnamed: 0,UNIQUEID,RIF_BINARY_PHENOTYPE,RIF_PHENOTYPE_QUALITY,RIF_MIC
0,0,site.02.subj.0001.lab.2014222001.iso.1,S,HIGH,<=0.06
1,1,site.02.subj.0002.lab.2014222005.iso.1,S,HIGH,<=0.06
2,2,site.02.subj.0004.lab.2014222010.iso.1,S,HIGH,0.12
3,3,site.02.subj.0005.lab.2014222011.iso.1,S,HIGH,0.12
4,4,site.02.subj.0006.lab.2014222013.iso.1,S,HIGH,0.12
...,...,...,...,...,...
12254,12254,site.20.subj.SCH8604398.lab.YA00134621.iso.1,S,HIGH,0.06
12255,12255,site.20.subj.SCH8604399.lab.YA00134971.iso.1,S,LOW,0.25
12256,12256,site.20.subj.SCH8608291.lab.YA00135762.iso.1,R,HIGH,>8
12257,12257,site.20.subj.SCH8608295.lab.YA00134623.iso.1,S,HIGH,0.06


In [ ]:
labels['phenotype'] = labels['RIF_BINARY_PHENOTYPE'].replace({'S': 1, 'R': 0})
y = np.array(labels['phenotype'])

In [ ]:
def load_and_concatenate_npy_files(npy_files):
    """Load and concatenate all .npy files."""
    arrays = np.load(npy_files, allow_pickle=True)
    if arrays.size == 0 or arrays.size == 1:
      return None
    concatenated_array = np.squeeze(arrays)
    return concatenated_array[0,:]

In [ ]:
import os
import fnmatch
import numpy as np

def find_npy_files(base_dir, pattern):
    """Recursively find all .npy files matching the pattern in base_dir."""
    npy_files = []
    for root, _, files in os.walk(base_dir):
        for file in files:
            if fnmatch.fnmatch(file, pattern):
                npy_files.append(os.path.join(root, file))
    return npy_files[0]

files_list = labels['UNIQUEID'].tolist()

In [ ]:
import csv

# Define the path to the CSV file
file_path = '/content/drive/MyDrive/npy_files_final.csv'

# Initialize an empty list to hold the CSV data
csv_data = []

# Open and read the CSV file
with open(file_path, newline='') as csvfile:
    csvreader = csv.reader(csvfile)
    for row in csvreader:
        # Append the entire row (which is a list of values) to csv_data
        csv_data.extend(row)

In [ ]:
csv_data[:10]

['/content/drive/MyDrive/BioE212/EVO/rif_embeddings_v1/embeds_1.0_singles/site_02/site.02.subj.0001.lab.2014222001.iso.1.npy',
 '/content/drive/MyDrive/BioE212/EVO/rif_embeddings_v1/embeds_1.0_singles/site_02/site.02.subj.0002.lab.2014222005.iso.1.npy',
 '/content/drive/MyDrive/BioE212/EVO/rif_embeddings_v1/embeds_1.0_singles/site_02/site.02.subj.0004.lab.2014222010.iso.1.npy',
 '/content/drive/MyDrive/BioE212/EVO/rif_embeddings_v1/embeds_1.0_singles/site_02/site.02.subj.0005.lab.2014222011.iso.1.npy',
 '/content/drive/MyDrive/BioE212/EVO/rif_embeddings_v1/embeds_1.0_singles/site_02/site.02.subj.0006.lab.2014222013.iso.1.npy',
 '/content/drive/MyDrive/BioE212/EVO/rif_embeddings_v1/embeds_1.0_singles/site_02/site.02.subj.0007.lab.2014222016.iso.1.npy',
 '/content/drive/MyDrive/BioE212/EVO/rif_embeddings_v1/embeds_1.0_singles/site_02/site.02.subj.0008.lab.2014222017.iso.1.npy',
 '/content/drive/MyDrive/BioE212/EVO/rif_embeddings_v1/embeds_1.0_singles/site_02/site.02.subj.0009.lab.2014222

In [ ]:
import tensorflow as tf

# Define a function to load and concatenate arrays in batches
def load_and_concatenate_batches(npy_files, batch_size):
    concatenated_batches = []
    for start_index in range(0, len(npy_files), batch_size):
        batch_files = npy_files[start_index:start_index+batch_size]
        batch_list = []
        for npy_file in batch_files:
          array = load_and_concatenate_npy_files(npy_file)
          if array is not None:
            batch_list.append(np.squeeze(array))
          else:
            print(npy_file)
        batch = np.stack(batch_list, axis=0)
        batch = np.squeeze(batch)
        np.save(str('/content/drive/MyDrive/rif_midembeddings_0527' + str(start_index) + "_arrayfinal.npy"), batch)
        concatenated_batches.append(batch)
        print(start_index)
    concat_batch = np.vstack(concatenated_batches)

    return concat_batch


batch_size = 100  # Adjust batch size based on available GPU memory
concatenated_array = load_and_concatenate_batches(csv_data, batch_size)


In [ ]:
nan_indices = np.isnan(y)
y = y[~nan_indices]
concatenated_array = concatenated_array[~nan_indices]
X_train, X_test, y_train, y_test = train_test_split(concatenated_array, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import torch

In [ ]:
# Train a logistic regression model
clf = LogisticRegression(multi_class='ovr')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
for i in range(10):
    print(f'Predicted: {y_pred[i]}, Actual: {y_test[i]}')

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score
cm = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = cm.ravel()

# Calculate sensitivity and specificity
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)

print(f'Confusion Matrix:\n{cm}')
print(f'Sensitivity (Recall): {sensitivity}')
print(f'Specificity: {specificity}')

In [ ]:
#PCA

import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

In [ ]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# Perform PCA
pca = PCA(n_components=2)
pca_result = pca.fit_transform(concatenated_array)

# Calculate the percent variation accounted for by each component
explained_variance = pca.explained_variance_ratio_ * 100

# Plot the PCA results
plt.figure(figsize=(10, 8))
scatter = plt.scatter(pca_result[:, 0], pca_result[:, 1], c=y, cmap='viridis', s=50, alpha=0.7)

# Adding a color bar
cbar = plt.colorbar(scatter)
cbar.set_label('Values')

# Annotate the axes with the percent variation
plt.xlabel(f'PCA Component 1 ({explained_variance[0]:.2f}% variance)')
plt.ylabel(f'PCA Component 2 ({explained_variance[1]:.2f}% variance)')
plt.title('PCA of Embeddings for RIF Resistance Prediction')
plt.show()

In [ ]:
'''
import numpy as np
from sklearn.decomposition import IncrementalPCA

# Assuming embeddings is a numpy array of shape (num_samples, original_dim)
n_components = 100  # Reduce to 2 dimensions
batch_size = 200  # Number of samples per batch

ipca = IncrementalPCA(n_components=n_components)

for batch in np.array_split(sample_resize, len(sample_resize) // batch_size):
    ipca.partial_fit(batch)

reduced_embeddings = ipca.transform(sample_resize)

print("Original shape:", sample_resize.shape)
print("Reduced shape:", reduced_embeddings.shape)

'''